<span style="color:red; font-weight: bold;">Background Metric:</span> The starting point is the flat FLRW (Friedmann–Lemaître–Robertson–Walker) metric, which represents a homogeneous and isotropic universe. In the absence of perturbations, this metric looks like:

$ds^2 = a(t)^2 \left( -d\tau^2 + dx^2 + dy^2 + dz^2 \right)$

$a(t)$ is the scale factor of the universe, and $x,y,z$ represent spatial coordinates.

<span style="color:red; font-weight: bold;">Perturbation:</span> The idea is to introduce small perturbations around this background metric to study how deviations evolve. In the Newtonian gauge, the metric is written as:
$ds^2 = a(t)^2 \left[ -(1 + 2\Phi(t, x)) d\tau^2 + (1 - 2\Psi(t, x)) \left( dx^2 + dy^2 + dz^2 \right) \right] $



$\Psi (t,x)$ are small scalar perturbations (gravitational potential) that represent the deviations from the homogeneous background.
These perturbations are assumed to be of first order in the small parameter 
$\epsilon$, i.e., linear perturbations.

<span style="color:red; font-weight: bold;">Gauge Choice:</span> The Newtonian gauge is often chosen because it allows for a simple interpretation of perturbations, with 
$\Phi$ representing the perturbation to the time-time component (gravitational potential) and 
$\Psi$ representing perturbations to the space-space components.

<span style="color:red; font-weight: bold;">Linear Perturbations:</span> These are the first-order perturbations in the metric, which are assumed to be small, i.e., 
$\Phi(t, x), \quad \Psi(t, x) \ll 1$. The goal is to solve the Einstein field equations to understand how these perturbations evolve in time.

<span style="color:red; font-weight: bold;">Einstein Equations:</span> The linearized Einstein equations are derived by plugging the perturbed metric into the Einstein-Hilbert action and keeping terms linear in the perturbations. This leads to equations for the perturbations 
$\Phi$ and $\Psi$, which describe how small inhomogeneities (such as gravitational waves, density fluctuations) grow over time.

<span style="color:red; font-weight: bold;">Application:</span> Newtonian gauge is widely used to study:

Cosmological perturbations: Fluctuations in the cosmic microwave background (CMB), the formation of large-scale structure in the universe, and the growth of density fluctuations.
Gravitational waves: Linear perturbations in the metric can also represent gravitational waves propagating through the spacetime.

In [11]:
import sympy as sym
from sympy import init_printing

# Initialize pretty printing for Jupyter
init_printing()

dim = 4
numcoor = sym.symarray('x', dim)  # u is a reserved symbol


# Newtonian Gauge metric
t, x, y, z, epsilon = sym.symbols('tau, x, y, z, epsilon')
coors = [t, x, y, z]
a, Phi, Psi = sym.symbols('a, Phi, Psi', cls=sym.Function)
metric = sym.diag(
    a(t)**2*(1 + 2*Phi(t, x, y, z)*epsilon),
    -a(t)**2*(1 - 2*Psi(t, x, y, z)*epsilon),
    -a(t)**2*(1 - 2*Psi(t, x, y, z)*epsilon),
    -a(t)**2*(1 - 2*Psi(t, x, y, z)*epsilon)
)

# Allocate space to save connections, Riemann tensor, and Ricci tensor
gam_down = sym.MutableDenseNDimArray(range(dim**3), shape=(dim, dim, dim))
gam_up = sym.MutableDenseNDimArray(range(dim**3), shape=(dim, dim, dim))
ricci_down = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))
ricci_mixed = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))
ricci_up = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))
einstein_down = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))
einstein_mixed = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))
einstein_up = sym.MutableDenseNDimArray(range(dim**2), shape=(dim, dim))

gdown = metric.subs([(coors[i], numcoor[i]) for i in range(dim)])
gup = gdown ** -1

detg = first_order(gdown.det())


def first_order(expr):
    return sym.series(expr, epsilon, 0, 2)

def first_order_2D(expr):
    for i in range(dim):
        for j in range(dim):
            expr[i, j] = first_order(expr[i, j])

def connection_down(i, j, k):
    return first_order((sym.diff(gdown[i, j], numcoor[k]) + sym.diff(gdown[i, k], numcoor[j]) - sym.diff(gdown[j, k], numcoor[i])) / 2)

def connection_up(i, j, k):
    gam = 0
    for l in range(dim):
        gam += gam_down[l, j, k] * gup[l, i]
    return first_order(sym.simplify(gam))

# compute connection \Gamma_{ijk}
for i in range(dim):
    for j in range(dim):
        for k in range(j+1):
            gam_down[i, j, k] = connection_down(i, j, k)
            if (j != k):
                gam_down[i, k, j] = gam_down[i, j, k]

# compute connection \Gamma^i_{jk}
for i in range(dim):
    for j in range(dim):
        for k in range(j+1):
            gam_up[i, j, k] = connection_up(i, j, k)
            if (j != k):
                gam_up[i, k, j] = gam_up[i, j, k]



# =================== compute Ricci tensor ==============================
def Riemann_tensor_up(i, j, k, l):  ## R^i_{  jkl}
    R = sym.diff(gam_up[i, j, k], numcoor[l]) - sym.diff(gam_up[i, j, l], numcoor[k])
    for m in range(dim):
        R += gam_up[i, m, l] * gam_up[m, j, k] - gam_up[i, m, k] * gam_up[m, j, l]
    return first_order(sym.simplify(R))

for i in range(dim):
    for j in range(i+1):
        ricci_down[i, j] = 0
        for k in range(dim):
            ricci_down[i, j] += Riemann_tensor_up(k, i, j, k)
        ricci_down[i, j] = first_order(sym.simplify(ricci_down[i, j]))
        if (i != j):
            ricci_down[j, i] = ricci_down[i, j]

for i in range(dim):
    for j in range(dim):
        ricci_mixed[i, j] = 0
        for k in range(dim):
            ricci_mixed[i, j] += gup[i, k] * ricci_down[k, j]
        ricci_mixed[i, j] = first_order(sym.simplify(ricci_mixed[i, j]))

for i in range(dim):
    for j in range(i+1):
        ricci_up[i, j] = 0
        for k in range(dim):
            ricci_up[i, j] += ricci_mixed[i, k] * gup[k, j]
        ricci_up[i, j] = first_order(sym.simplify(ricci_up[i, j]))
        if (i != j):
            ricci_up[j, i] = ricci_up[i, j]

## ------------------ now Ricci tensor is saved -----------------------

## =========================== compute Ricci scalar ====================
ricci_scalar = 0
for k in range(dim):
    for l in range(k+1):
        if (gup[k, l] != 0):
            if (k == l):
                ricci_scalar += ricci_down[k, l] * gup[k, l]
            else:
                ricci_scalar += 2 * ricci_down[k, l] * gup[k, l]
ricci_scalar = first_order(sym.simplify(ricci_scalar))

# ======================= compute Einstein tensor ==================
for i in range(dim):
    for j in range(i+1):
        einstein_down[i, j] = first_order(ricci_down[i, j] - ricci_scalar / 2 * gdown[i, j])
        einstein_up[i, j] = first_order(ricci_up[i, j] - ricci_scalar / 2 * gup[i, j])
        if (i != j):
            einstein_down[j, i] = einstein_down[i, j]
            einstein_up[j, i] = einstein_up[i, j]
for i in range(dim):
    for j in range(dim):
        if (i == j):
            einstein_mixed[i, j] = first_order(ricci_mixed[i, j] - ricci_scalar / 2)
        else:
            einstein_mixed[i, j] = ricci_mixed[i, j]

# Print each tensor/scalar separately in LaTeX format
# Print Einstein Tensor (G^i_j)
print("\n## Einstein Tensor (G^i_j):")
for i in range(dim):
    for j in range(dim):
        latex_str = sym.latex(einstein_mixed[i, j])
        print(f"G^{{{i}}}_{{{j}}} = {latex_str}")  # Einstein Tensor components
        print()  # Adding space between iterations

# Add some space between sections for clarity
print("\n" + "#" * 50 + "\n")

# Print Ricci Scalar (R)
print("## Ricci Scalar (R):")
latex_str = sym.latex(ricci_scalar)
print(f"R = {latex_str}")  # Ricci Scalar value
print()  # Adding space after printing Ricci Scalar

# Add some space between sections for clarity
print("\n" + "#" * 50 + "\n")

# Print Ricci Tensor (R^i_j)
print("## Ricci Tensor (R^i_j):")
for i in range(dim):
    for j in range(dim):
        latex_str = sym.latex(ricci_up[i, j])
        print(f"R^{{{i}}}_{{{j}}} = {latex_str}")  # Ricci Tensor components
        print()  # Adding space between iterations



## Einstein Tensor (G^i_j):
G^{0}_{0} = \frac{3 \left(\frac{d}{d x_{0}} a{\left(x_{0} \right)}\right)^{2}}{a^{4}{\left(x_{0} \right)}} + \epsilon \left(- \frac{3 \cdot \left(2 \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} - 2 \Psi{\left(x_{0},x_{1},x_{2},x_{3} \right)}\right) \frac{d^{2}}{d x_{0}^{2}} a{\left(x_{0} \right)}}{a^{3}{\left(x_{0} \right)}} + \frac{\frac{\partial^{2}}{\partial x_{1}^{2}} \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} + \frac{\partial^{2}}{\partial x_{2}^{2}} \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} + \frac{\partial^{2}}{\partial x_{3}^{2}} \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} + 3 \frac{\partial^{2}}{\partial x_{0}^{2}} \Psi{\left(x_{0},x_{1},x_{2},x_{3} \right)}}{a^{2}{\left(x_{0} \right)}} + \frac{6 \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} \frac{d^{2}}{d x_{0}^{2}} a{\left(x_{0} \right)}}{a^{3}{\left(x_{0} \right)}} - \frac{6 \Phi{\left(x_{0},x_{1},x_{2},x_{3} \right)} \left(\frac{d}{d x_{0}} a{\left(x_{0} \right)}\right)^{2}}{a^{4}{\left(x_{0} \r